In [105]:
from typing_extensions import Union
import json
from pydantic import BaseModel


class Thing(BaseModel):
    name: str
    description: str | None = None


class Bag(BaseModel):
    color: str
    things: list[Thing]


class Person(BaseModel):
    first_name: str
    last_name: str
    middle_name: Union[str, None] = None
    age: int | None = None
    bag: Bag | None = None


In [106]:
print(json.dumps(Person.model_json_schema(), indent=4))

{
    "$defs": {
        "Bag": {
            "properties": {
                "color": {
                    "title": "Color",
                    "type": "string"
                },
                "things": {
                    "items": {
                        "$ref": "#/$defs/Thing"
                    },
                    "title": "Things",
                    "type": "array"
                }
            },
            "required": [
                "color",
                "things"
            ],
            "title": "Bag",
            "type": "object"
        },
        "Thing": {
            "properties": {
                "name": {
                    "title": "Name",
                    "type": "string"
                },
                "description": {
                    "anyOf": [
                        {
                            "type": "string"
                        },
                        {
                            "type": "null"
                       

In [107]:
from pydantic import create_model

def remove_field(base_model: type[BaseModel], field_to_remove: str) -> type[BaseModel]:
    fields = {k: (v.annotation, v.default) for k, v in base_model.model_fields.items() if k != field_to_remove}

    new_model = create_model(
        f'{base_model.__name__}Copy',
        **fields,
    )

    return new_model

PersonCopy = remove_field(Person, 'last_name')

print(json.dumps(PersonCopy.model_json_schema(), indent=4))


{
    "$defs": {
        "Bag": {
            "properties": {
                "color": {
                    "title": "Color",
                    "type": "string"
                },
                "things": {
                    "items": {
                        "$ref": "#/$defs/Thing"
                    },
                    "title": "Things",
                    "type": "array"
                }
            },
            "required": [
                "color",
                "things"
            ],
            "title": "Bag",
            "type": "object"
        },
        "Thing": {
            "properties": {
                "name": {
                    "title": "Name",
                    "type": "string"
                },
                "description": {
                    "anyOf": [
                        {
                            "type": "string"
                        },
                        {
                            "type": "null"
                       

In [109]:
from types import UnionType
from typing_extensions import get_origin, get_args, Union


def remove_fields(base_model: type[BaseModel], excludes: dict) -> type[BaseModel]:
    def process_excludes(fields, exclude_dict):
        new_fields = {}
        for field_name, field_info in fields.items():
            print(f'\n\n{field_name}')
            print(type(field_info.annotation))
            annotation_origin = get_origin(field_info.annotation)
            if annotation_origin is Union or annotation_origin is UnionType:
                annotation_args = get_args(field_info.annotation)
                inner_annotation = [arg for arg in annotation_args if arg is not None][0]
                print('UNION')
            else:
                inner_annotation = field_info.annotation
                print('NOT UNION')
            print(inner_annotation)

            print(issubclass(inner_annotation, BaseModel))

            if field_name not in exclude_dict:
                # If the field is not explicitly excluded, include it
                new_fields[field_name] = (field_info.annotation, field_info.default_factory or field_info.default)
            elif isinstance(exclude_dict[field_name], dict):
                # If the field is a nested dictionary, process recursively
                if hasattr(field_info.annotation, 'model_fields'):
                    # Assuming the annotation is another Pydantic model
                    sub_model = field_info.annotation
                    print(sub_model)
                    new_sub_model = remove_fields(sub_model, exclude_dict[field_name])
                    new_fields[field_name] = (new_sub_model, field_info.default_factory or field_info.default)
                else:
                    # If the annotation is not a Pydantic model, include it as is
                    new_fields[field_name] = (field_info.annotation, field_info.default_factory or field_info.default)
        return new_fields

    # Create a new dictionary excluding the specified fields
    fields = process_excludes(base_model.model_fields, excludes)

    # Use create_model to dynamically create a new model with the filtered fields
    new_model = create_model(
        f'{base_model.__name__}Copy',
        **fields
    )

    return new_model

# Define the nested exclude structure
exclude_structure = {
    'last_name': True,
    'bag': {'color': True}
}

# Remove specified fields from the Person class
PersonCopy = remove_fields(Person, exclude_structure)

# Print the JSON schema of the new model
# print(json.dumps(PersonCopy.schema(), indent=4))



first_name
<class 'type'>
NOT UNION
<class 'str'>
False


last_name
<class 'type'>
NOT UNION
<class 'str'>
False


middle_name
<class 'typing._UnionGenericAlias'>
UNION
<class 'str'>
False


age
<class 'types.UnionType'>
UNION
<class 'int'>
False


bag
<class 'types.UnionType'>
UNION
<class '__main__.Bag'>
True
